In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import librosa
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from IPython.display import SVG, Image
from livelossplot.inputs.tf_keras import PlotLossesCallback
import tensorflow as tf
print("Tensorflow version:", tf.__version__)

# Import the necessary libraries
from PIL import Image
from numpy import asarray
import numpy as np
import os
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os
from keras.utils import np_utils
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
# calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
from sklearn.metrics import f1_score
print('all imported')

Tensorflow version: 2.6.0
all imported


In [2]:
path_dir = 'data/'

In [3]:
def read_sound(file_path):
    y, sr = librosa.load(file_path, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
    return librosa.feature.melspectrogram(y=y, sr=sr)
read_sound('data/neg/AnyConv.com__neg-0421-083-cough-m-53-0.wav')

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [4]:
listFiles = os.listdir('data/neg/')
print(listFiles[:5])

['AnyConv.com__neg-0421-083-cough-m-53-0.wav', 'AnyConv.com__neg-0421-083-cough-m-53-1.wav', 'AnyConv.com__neg-0421-083-cough-m-53-11.wav', 'AnyConv.com__neg-0421-083-cough-m-53-12.wav', 'AnyConv.com__neg-0421-083-cough-m-53-13.wav']


In [5]:
#load Benign
listCancer = []
listResults = []
for elem in listFiles:
    img = read_sound('data/neg/'+elem)
    listCancer += [img]
    listResults += [np.array([1])]
print(listCancer[:3])
print(img.shape)

[array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), array([[1.5383340e-04, 5.0781910e-05, 1.4765496e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.8477437e-04, 6.5516302e-05, 1.4249467e-06, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [3.9289967e-05, 1.6717848e-05, 8.2170897e-07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [1.2430817e-08, 3.2063012e-09, 8.9009154e-11, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.2183010e-08, 3.

In [6]:
listFiles = os.listdir('data/pos/')
print(listFiles[:5])
for elem in listFiles:
    img = read_sound('data/pos/'+elem)
    listResults += [np.array([0])]
    listCancer += [img]

['AnyConv.com__pos-0421-084-cough-m-50-1.wav', 'AnyConv.com__pos-0421-084-cough-m-50-2.wav', 'AnyConv.com__pos-0421-084-cough-m-50-3.wav', 'AnyConv.com__pos-0421-084-cough-m-50-5.wav', 'AnyConv.com__pos-0421-084-cough-m-50-6.wav']


In [68]:
# Setting LR for different number of Epochs
def lr_schedule(epoch):
    lr = 1
    if epoch > 50:
        lr *= 0.04
    lr /= epoch + 100
    print('Learning rate: ', lr)
    return lr

# Basic ResNet Building Block
def resnet_layer(inputs, conv_first = False,
				num_filters = 16,
				kernel_size = 3,
				strides = 1,
				activation ='relu',
				batch_normalization = True):
	conv = Conv2D(num_filters,
				kernel_size = kernel_size,
				strides = strides,
				padding ='same',
				kernel_initializer ='he_normal',
				kernel_regularizer = l2(1e-4))

	x = inputs
	if conv_first:
		x = conv(x)
		if batch_normalization:
			x = BatchNormalization()(x)
		if activation is not None:
			x = Activation(activation)(x)
	else:
		if batch_normalization:
			x = BatchNormalization()(x)
		if activation is not None:
			x = Activation(activation)(x)
		x = conv(x)
	return x

# ResNet V2 architecture
def resnet_v2(input_shape, depth, num_classes = 10):
	if (depth - 2) % 9 != 0:
		raise ValueError('depth should be 9n + 2 (eg 56 or 110 in [b])')
	# Start model definition.
	num_filters_in = 16
	num_res_blocks = int((depth - 2) / 9)

	inputs = Input(shape = input_shape)
	# v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
	x = resnet_layer(inputs = inputs,num_filters = num_filters_in,conv_first = True)

	# Instantiate the stack of residual units
	for stage in range(3):
		for res_block in range(num_res_blocks):
			activation = 'relu'
			batch_normalization = True
			strides = 1
			if stage == 0:
				num_filters_out = num_filters_in * 4
				if res_block == 0: # first layer and first stage
					activation = None
					batch_normalization = False
			else:
				num_filters_out = num_filters_in * 2
				if res_block == 0: # first layer but not first stage
					strides = 2 # downsample

			# bottleneck residual unit
			y = resnet_layer(inputs = x,
							num_filters = num_filters_in,
							kernel_size = 1,
							strides = strides,
							activation = activation,
							batch_normalization = batch_normalization)
			y = resnet_layer(inputs = y,
							num_filters = num_filters_in,
							conv_first = False)
			y = resnet_layer(inputs = y,
							num_filters = num_filters_out,
							kernel_size = 1,
							conv_first = False)
			if res_block == 0:
				# linear projection residual shortcut connection to match
				# changed dims
				x = resnet_layer(inputs = x,
								num_filters = num_filters_out,
								kernel_size = 1,
								strides = strides,
								activation = None,
								batch_normalization = False)
			x = keras.layers.add([x, y])

		num_filters_in = num_filters_out

	# Add classifier on top.
	# v2 has BN-ReLU before Pooling
	x = BatchNormalization()(x)
	x = Activation('relu')(x)
	x = AveragePooling2D(pool_size = 2)(x)
	y = Flatten()(x)
	outputs = Dense(num_classes,
					activation ='softmax',
					kernel_initializer ='he_normal')(y)

	# Instantiate model.
	model = Model(inputs = inputs, outputs = outputs)
	return model

In [69]:
print(len(listCancer))

97


In [110]:
x_train=[]
y_train=[]
accs=[]
import random
def main(pz):
    for i in range(3):
        per = np.random.permutation(len(listCancer))
    ln = int(len(listCancer) * 0.7)
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    positions = []
    st = 0
    dr = len(listCancer) - 1
    ln = int(len(listCancer) * 0.25)
    per = np.random.permutation(54)
    while ln>0:
        positions += [per[st]]
        st+=1
        ln-=1
      
    st = 0
    ln = int(len(listCancer) * 0.3)
    per = np.random.permutation(43)
    while ln>0:
        positions += [per[st] + 54]
        st+=1
        ln-=1
        #positions += [per[i]]
    for i in range(len(listCancer)):
        if i in positions:
            x_train += [listCancer[i]]
            y_train += [listResults[i]]
        else:
            x_test += [listCancer[i]]
            y_test += [listResults[i]]
            
    ln = len(x_test) // 2
    per = np.random.permutation(len(x_test))
    x_aux = []
    y_aux = []
    for h in range(len(per)):
        for z in range(len(per)):
            if per[z] == h:
                x_aux+=[x_test[z]]
                y_aux+=[y_test[z]]
    x_test = x_aux
    y_test = y_aux
    print(x_test)
    
    x_valid = x_test
    y_valid = y_test
    
    x_test = x_test
    y_test = y_test
    
    
    x_train = np.array(x_train)
    x_train = x_train.reshape(len(x_train), 40, 16, 19)
    y_train = np.array(y_train)
    

    x_test = np.array(x_test)
    x_test = x_test.reshape(len(x_test), 40, 16, 19)
    y_test = np.array(y_test)
    
    x_valid = np.array(x_valid)
    x_valid = x_valid.reshape(len(x_valid), 40, 16, 19)
    y_valid = np.array(y_valid)
    
    
    batch_size = 32 # original ResNet paper uses batch_size = 128 for training
    epochs = 10
    data_augmentation = True
    num_classes = 10

    # Data Preprocessing
    subtract_pixel_mean = True
    n = 3

    # Select ResNet Version
    version = 2

    # Computed depth of
    if version == 1:
        depth = n * 6 + 2
    elif version == 2:
        depth = n * 9 + 2

    # Model name, depth and version
    model_type = 'ResNet % dv % d' % (depth, version)

    # use the data
    print(x_train[:3])
    print(y_train[:3])
    print(type(x_train[0]))
    print(type(x_train))
    print(type(y_train[0]))
    print(type(y_train))

    # Input image dimensions.
    input_shape = x_train.shape[1:]
    print(input_shape)

    # Normalize data.
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    x_valid = x_test.astype('float32') / 255

    # If subtract pixel mean is enabled
    if subtract_pixel_mean:
        x_train_mean = np.mean(x_train, axis = 0)
        x_train -= x_train_mean
        x_test -= x_train_mean

    # Print Training and Test Samples
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print('y_train shape:', y_train.shape)

    # Convert class vectors to binary class matrices.
    y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)
    y_valid = keras.utils.np_utils.to_categorical(y_valid, num_classes)
    
    
    model = resnet_v2(input_shape = input_shape, depth = depth)

    model.compile(loss ='categorical_crossentropy',
                optimizer = Adam(learning_rate = lr_schedule(0)),
                metrics =['accuracy'])
    model.summary()
    print(model_type)

    # Prepare model model saving directory.
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = 'cifar10_% s_model.{epoch:03d}.h5' % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(filepath = filepath,
                                monitor ='val_acc',
                                verbose = 1,
                                save_best_only = True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(factor = np.sqrt(0.1),
                                cooldown = 0,
                                patience = 5,
                                min_lr = 0.5e-6)

    callbacks = [checkpoint, lr_reducer, lr_scheduler]

    # Run training, with or without data augmentation.
    if not data_augmentation:
        print('Not using data augmentation.')
        model.fit(x_train, y_train,
                batch_size = batch_size,
                epochs = epochs,
                validation_data =(x_test, y_test),
                shuffle = True,
                callbacks = callbacks)
    else:
        print('Using real-time data augmentation.')
        # This will do preprocessing and realtime data augmentation:
        datagen = ImageDataGenerator(
            # set input mean to 0 over the dataset
            featurewise_center = False,
            # set each sample mean to 0
            samplewise_center = False,
            # divide inputs by std of dataset
            featurewise_std_normalization = False,
            # divide each input by its std
            samplewise_std_normalization = False,
            # apply ZCA whitening
            zca_whitening = False,
            # epsilon for ZCA whitening
            zca_epsilon = 1e-06,
            # randomly rotate images in the range (deg 0 to 180)
            rotation_range = 0,
            # randomly shift images horizontally
            width_shift_range = 0.1,
            # randomly shift images vertically
            height_shift_range = 0.1,
            # set range for random shear
            shear_range = 0.,
            # set range for random zoom
            zoom_range = 0.,
            # set range for random channel shifts
            channel_shift_range = 0.,
            # set mode for filling points outside the input boundaries
            fill_mode ='nearest',
            # value used for fill_mode = "constant"
            cval = 0.,
            # randomly flip images
            horizontal_flip = True,
            # randomly flip images
            vertical_flip = False,
            # set rescaling factor (applied before any other transformation)
            rescale = None,
            # set function that will be applied on each input
            preprocessing_function = None,
            # image data format, either "channels_first" or "channels_last"
            data_format = None,
            # fraction of images reserved for validation (strictly between 0 and 1)
            validation_split = 0.1)

        # Compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied).
        print('------------------')
        print(x_train)
        datagen.fit(x_train)

        # Fit the model on the batches generated by datagen.flow().
        model.fit_generator(datagen.flow(x_train, y_train, batch_size = batch_size),
                            validation_data =(x_valid, y_valid),
                            epochs = 100, verbose = 1, workers = 4,
                            callbacks = callbacks)

    # Score trained model.
    scores = model.evaluate(x_test, y_test, verbose = 1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    ret = model.predict(x_test)
    result = []
    pred = []
    for a,b in zip(ret, y_test):
        pred+=[a.argmax(axis=-1)]
        result+=[b.argmax(axis=-1)]
    print(pred)
    print(result)
    precision = precision_score(result, pred, average='weighted')
    recall = recall_score(result, pred, average='weighted')
     # calculates f1 for 1:100 dataset with 95tp, 5fn, 55fp
    fmeasure = f1_score(result, pred, average='weighted')
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 Score: ', fmeasure)
    #precisions += [precision]
    #recalls += [recall]
    #model.save('saved_models/resNetV'+pz+'.h5')
    return [scores[1], precision, recall, fmeasure]

In [111]:
accs=[]
precisions = []
recalls = []
fmeasures = []
while True:
    if len(accs) > 9:
        break
    ret = main(i)
    if ret[0] < 0.54:
        continue
    accs+=[ret[0]]
    precisions+=[ret[1]]
    recalls+=[ret[2]]
    fmeasures+=[ret[3]]
print(accs)
print(precisions)
print(recalls)
print(fmeasures)

[array([[2.3606790e-05, 5.5208830e-06, 3.3270757e-07, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [6.0420294e-05, 1.8507157e-05, 1.1242225e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [9.8333505e-05, 6.1406412e-05, 3.6835598e-05, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [4.3888844e-09, 1.1880769e-09, 1.3228339e-10, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.2975295e-09, 1.3415808e-09, 3.0002133e-11, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [5.4958016e-09, 1.3766267e-09, 3.5165349e-12, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32), array([[1.99750648e-04, 4.66674319e-05, 4.10542043e-07, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.08512540e-04, 1.20408018e-04, 6.17323212e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.05901252e-04, 1.18306634e-04, 3.97949407e-05, ...,
 

Learning rate:  0.01
Model: "model_121"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_122 (InputLayer)          [(None, 40, 16, 19)] 0                                            
__________________________________________________________________________________________________
conv2d_3751 (Conv2D)            (None, 40, 16, 16)   2752        input_122[0][0]                  
__________________________________________________________________________________________________
batch_normalization_3388 (Batch (None, 40, 16, 16)   64          conv2d_3751[0][0]                
__________________________________________________________________________________________________
activation_3388 (Activation)    (None, 40, 16, 16)   0           batch_normalization_3388[0][0]   
_____________________________________________________________________

2/2 [==============================] - 6s 1s/step - loss: 2.9096 - accuracy: 0.2200 - val_loss: 20.0569 - val_accuracy: 0.7234
Epoch 2/100
Learning rate:  0.009900990099009901
2/2 [==============================] - 1s 517ms/step - loss: 3.1717 - accuracy: 0.4000 - val_loss: 779.8338 - val_accuracy: 0.2766
Epoch 3/100
Learning rate:  0.00980392156862745
2/2 [==============================] - 1s 708ms/step - loss: 1.4156 - accuracy: 0.4800 - val_loss: 2036.5077 - val_accuracy: 0.2766
Epoch 4/100
Learning rate:  0.009708737864077669
2/2 [==============================] - 1s 480ms/step - loss: 1.8622 - accuracy: 0.6000 - val_loss: 1421.1907 - val_accuracy: 0.2766
Epoch 5/100
Learning rate:  0.009615384615384616
2/2 [==============================] - 1s 688ms/step - loss: 1.8053 - accuracy: 0.6000 - val_loss: 729.0004 - val_accuracy: 0.2766
Epoch 6/100
Learning rate:  0.009523809523809525
2/2 [==============================] - 1s 660ms/step - loss: 1.4965 - accuracy: 0.6000 - val_loss: 187.

2/2 [==============================] - 1s 507ms/step - loss: 1.3838 - accuracy: 0.6200 - val_loss: 1.2782 - val_accuracy: 0.2766
Epoch 34/100
Learning rate:  0.007518796992481203
2/2 [==============================] - 1s 487ms/step - loss: 1.1702 - accuracy: 0.5000 - val_loss: 1.2034 - val_accuracy: 0.7234
Epoch 35/100
Learning rate:  0.007462686567164179
2/2 [==============================] - 1s 664ms/step - loss: 1.6367 - accuracy: 0.4800 - val_loss: 1.1944 - val_accuracy: 0.7234
Epoch 36/100
Learning rate:  0.007407407407407408
2/2 [==============================] - 1s 485ms/step - loss: 1.2343 - accuracy: 0.4800 - val_loss: 1.2452 - val_accuracy: 0.7234
Epoch 37/100
Learning rate:  0.007352941176470588
2/2 [==============================] - 1s 805ms/step - loss: 1.1522 - accuracy: 0.5600 - val_loss: 1.3248 - val_accuracy: 0.2766
Epoch 38/100
Learning rate:  0.0072992700729927005
2/2 [==============================] - 1s 632ms/step - loss: 1.2397 - accuracy: 0.6000 - val_loss: 1.382

2/2 [==============================] - 1s 593ms/step - loss: 1.0521 - accuracy: 0.6200 - val_loss: 1.2231 - val_accuracy: 0.2766
Epoch 66/100
Learning rate:  0.00024242424242424242
2/2 [==============================] - 1s 616ms/step - loss: 1.0454 - accuracy: 0.6200 - val_loss: 1.2209 - val_accuracy: 0.2766
Epoch 67/100
Learning rate:  0.00024096385542168676
2/2 [==============================] - 2s 1s/step - loss: 1.0520 - accuracy: 0.6200 - val_loss: 1.2190 - val_accuracy: 0.2766
Epoch 68/100
Learning rate:  0.00023952095808383233
2/2 [==============================] - 1s 616ms/step - loss: 1.0739 - accuracy: 0.6000 - val_loss: 1.2163 - val_accuracy: 0.2766
Epoch 69/100
Learning rate:  0.0002380952380952381
2/2 [==============================] - 1s 633ms/step - loss: 1.0465 - accuracy: 0.6200 - val_loss: 1.2144 - val_accuracy: 0.2766
Epoch 70/100
Learning rate:  0.00023668639053254438
2/2 [==============================] - 1s 844ms/step - loss: 1.0438 - accuracy: 0.6200 - val_loss: 

...

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Learning rate:  0.01
Model: "model_122"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_123 (InputLayer)          [(None, 40, 16, 19)] 0                                            
__________________________________________________________________________________________________
conv2d_3782 (Conv2D)            (None, 40, 16, 16)   2752        input_123[0][0]                  
__________________________________________________________________________________________________
batch_normalization_3416 (Batch (None, 40, 16, 16)   64          conv2d_3782[0][0]                
__________________________________________________________________________________________________
activation_3416 (Activation)    (None, 40, 16, 16)   0           batch_normalization_3416[0][0]   
_____________________________________________________________________

c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\image_data_generator.py:938: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (50, 40, 16, 19) (19 channels).
  warnings.warn(
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras_preprocessing\image\numpy_array_iterator.py:129: UserWarning: NumpyArrayIterator is set to use the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3, or 4 channels on axis 3. However, it was passed an array with shape (50, 40, 16, 19) (19 channels).
  warnings.warn('NumpyArrayIterator is set to use the '
c:\users\patcas rares\appdata\local\programs\python\python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and

1/2 [==============>...............] - ETA: 4s - loss: 2.8964 - accuracy: 0.0938

KeyboardInterrupt: 

In [ ]:
print(len(accs))
print('accs: ', accs)
print('precisions: ', precisions)
print('recalls: ', recalls)
print('fmeasures: ', fmeasures)

In [27]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.4541222062044867, 0.6489080939358943)

In [23]:
model.predict(x_test)

NameError: name 'model' is not defined

In [1]:
accs = [0.26829269528388977, 0.41860464215278625, 0.4651162922382355, 0.5555555820465088, 0.5116279125213623, 0.4285714328289032, 0.44999998807907104, 0.39534884691238403, 0.6666666865348816, 0.5454545617103577]
precisions = [0.07198096371207614, 0.1752298539751217, 0.2163331530557058, 0.7575757575757577, 0.7558139534883721, 0.1986062717770035, 0.20249999999999999, 0.15630070308274743, 0.44444444444444436, 0.29752066115702475]
recalls = [0.2682926829268293, 0.4186046511627907, 0.46511627906976744, 0.5555555555555556, 0.5116279069767442, 0.42857142857142855, 0.45, 0.3953488372093023, 0.6666666666666666, 0.5454545454545454]
fmeasures = [0.11350844277673547, 0.24704536789935186, 0.29531192321889993, 0.4188948306595365, 0.3700707785642063, 0.27142857142857146, 0.2793103448275862, 0.22403100775193796, 0.5333333333333334, 0.38502673796791437]

In [94]:
import numpy as np
import scipy.stats as st

#define sample data
data = accs

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.6539882966412536, 0.6943988016711244)

In [95]:
#define sample data
data = precisions

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.49031273655309493, 0.6830269359623643)

In [96]:
#define sample data
data = recalls

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.6539882917530159, 0.6943988050211779)

In [97]:
#define sample data
data = fmeasures

#create 95% confidence interval for population mean weight
st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))

(0.5504964410956525, 0.5997834952867505)